# Revtsov HW1

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp

(CVXPY) Apr 04 05:50:37 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


In [2]:
# Read the CSV data
labels = np.loadtxt('hw1_labels.csv', dtype=str)
mean = np.loadtxt('hw1_mean.csv', dtype=float)
stdev = np.loadtxt('hw1_std.csv', dtype=float)
corr = np.loadtxt('hw1_corr.csv', dtype=float)

In [3]:
def solve_min_var(mean, stdev, corr, w, constraints, labels=labels):
    """
    Helper function that will be used throughout the homework
    """
    # calculate covariance from correlations and standard deviations
    cov = np.diag(stdev) @ corr @ np.diag(stdev)
    
    # minimize variance of portfolio
    obj = cp.Minimize(cp.quad_form(w, cov))
    
    prob = cp.Problem(
        objective=obj,
        constraints=constraints,
    )
    
    prob.solve(solver=cp.ECOS)
    assert prob.status == 'optimal'

    p_var = w.value @ cov @ w.value.T
    p_risk = np.sqrt(p_var)
    p_ret = w.value @ mean
    print('\nPortfolio Weights:\n')
    print(pd.Series(index=labels, data=np.round(w.value * 1e2, 2), name='Weight'))
    print(f'\nPortfolio risk: {round(p_risk * 1e2, 2)}%')
    print(f'Portfolio return: {round(p_ret * 1e2, 2)}%')

## Problem 1
The constraints to problem 1 are defined below
- Weights sum to 1
- All weights are non-negative
- Weight to Lottery should be zero (I'm including the Lottery asset class here so I don't need to modify the inputs later)
- Weights to all other assets should be at least 2%
- Expected portfolio return at least 6% (the minimum variance problem should solve to exactly 6%)

In [4]:
# define the vector we're solving
w = cp.Variable(len(mean))

constraints = [
    # sum of all weights is one
    cp.sum(w) == 1,
    # all weights non-negative
    w >= 0,
    # force Lottery to 0
    w[-1] == 0,
    # weight in each asset should be at least 2% (excluding Lottery)
    w[:-1] >= 0.02,
    # set the expected return
    w @ mean >= 0.06,
]

The weights, portfolio risk, and portfolio return are printed out below. You can see that the expected return is 6%. Since the objective is minimum variance it makes sense that large portion of the allocation is in the two lower risk bond asset classes. The allocation to EmMkt also makes sense since that asset class has the highest return to risk ratio from the asset classes that are least correlated to bonds.

In [5]:
solve_min_var(mean, stdev, corr, w, constraints)


Portfolio Weights:

USLC        2.00
NUSDev      2.00
EmMkt      19.36
REIT        4.30
HY         32.93
Agg        39.41
Lottery     0.00
Name: Weight, dtype: float64

Portfolio risk: 7.88%
Portfolio return: 6.0%


## Problem 2
The constraints to problem 2 are defined below
- Weights sum to 1
- Weight to Lottery should be zero

In [6]:
# define the vector we're solving
w = cp.Variable(len(mean))

constraints2 = [
    # sum of all weights is one
    cp.sum(w) == 1,
    # force Lottery to 0
    w[-1] == 0,
]

The weights, portfolio risk, and portfolio return are printed out below. Again, large long positions in bonds. About net flat across the 3 equity assets, likely due to the high correlation between them.

In [7]:
solve_min_var(mean, stdev, corr, w, constraints2)


Portfolio Weights:

USLC      -15.02
NUSDev     19.26
EmMkt      -4.92
REIT        3.33
HY         32.28
Agg        65.08
Lottery     0.00
Name: Weight, dtype: float64

Portfolio risk: 5.22%
Portfolio return: 4.46%


## Problem 3
The constraints to problem 3 are defined below
- Weights sum to 1
- All weights are non-negative
- Weights non-Lottery assets should be at least 2%
- Expected portfolio return at least 6% (the minimum variance problem should solve to exactly 6%)

In [8]:
# define the vector we're solving
w = cp.Variable(len(mean))

constraints3 = [
    # sum of all weights is one
    cp.sum(w) == 1,
    # all weights non-negative
    w >= 0,
    # weight in each asset should be at least 2% (excluding Lottery)
    w[:-1] >= 0.02,
    # set the expected return
    w @ mean >= 0.06,
]

The weights, portfolio risk, and portfolio return are printed out below. There's nothing allocated to Lottery and the results are identical to Problem 1.

In [9]:
solve_min_var(mean, stdev, corr, w, constraints3)


Portfolio Weights:

USLC        2.00
NUSDev      2.00
EmMkt      19.36
REIT        4.30
HY         32.93
Agg        39.41
Lottery     0.00
Name: Weight, dtype: float64

Portfolio risk: 7.88%
Portfolio return: 6.0%


## Problem 4
We are given log-normal statistics and need to convert them to normal distribution. Using the formalus on slide 6 of the Portfolio Calculations powerpoint we arrive at the below. 

In [10]:
stdev_norm = np.sqrt(np.log((stdev**2)/((1+mean)**2)+1))

In [11]:
mean_norm = np.log(1+mean) - (stdev_norm**2)/2

In [12]:
covar = np.diag(stdev) @ corr @ np.diag(stdev)

In [13]:
covar_norm = np.log(1 + covar / np.outer(1+mean, 1+mean))

#### Standard Deviation

In [14]:
np.round(stdev_norm[0:-1], 4)

array([0.1655, 0.1579, 0.2157, 0.1836, 0.0666, 0.0574])

#### Mean

In [15]:
np.round(mean_norm[0:-1], 4)

array([0.0633, 0.0552, 0.0721, 0.0601, 0.0466, 0.0424])

#### Covariance

In [16]:
np.round(covar_norm[0:-1, 0:-1], 4)

array([[0.0274, 0.0236, 0.0287, 0.0244, 0.0044, 0.0019],
       [0.0236, 0.0249, 0.0307, 0.0233, 0.0042, 0.0009],
       [0.0287, 0.0307, 0.0465, 0.0279, 0.0044, 0.0013],
       [0.0244, 0.0233, 0.0279, 0.0337, 0.0037, 0.0011],
       [0.0044, 0.0042, 0.0044, 0.0037, 0.0044, 0.0015],
       [0.0019, 0.0009, 0.0013, 0.0011, 0.0015, 0.0033]])

## Problem 5
Didn't need to run an optimization but this is an interesting scenario. 

#### Part a
The weights of the uncertainty-free position are below, 25% USLC and 75% Agg. Both weights are positive since the correlation of the positions is -1. 

#### Part b
The uncertainty free rate of return is 5.37%.

In [17]:
mask = (labels == 'USLC') | (labels == 'Agg')

# define the vector we're solving
w = cp.Variable(sum(mask))

constraints5 = [
    # sum of all weights is one
    cp.sum(w) == 1,
]

solve_min_var(mean=mean[mask], stdev=stdev[mask], corr=np.array([[1, -1], [-1, 1]]), w=w, constraints=constraints5, labels=labels[mask])


Portfolio Weights:

USLC    25.0
Agg     75.0
Name: Weight, dtype: float64

Portfolio risk: 0.0%
Portfolio return: 5.37%


## Problem 6
The portfolio variance in this problem is given by

$$
    \sigma_{p}^2 = w_1^2 \sigma_1^2 + 2 w_1 w_2 \sigma_1 \sigma_2 \rho_{1, 2} + 2 w_1 w_3 \sigma_1 \sigma_3 \rho_{1, 3} + w_2^2 \sigma_2^2 + 2 w_2 w_3 \sigma_2 \sigma_3 \rho_{2, 3} + w_3^2 \sigma_3^2
$$

#### Part a
The standard deviations are below since the variances of the positions are given. 

In [18]:
stdev6 = np.sqrt([0.0289, 0.0225, 0.0144])
print(stdev6)

[0.17 0.15 0.12]


#### Part b
The term XXX has a range of values because we don't know the correlations of the assets. The term will be largest when correlation is 1 and smallest when it is -1. The term is calculated with

$$2 \sigma_2 \sigma_3 \rho_{2, 3}$$

In [19]:
xxx_range = [
    2 * stdev6[1] * stdev6[2] * 1,
    2 * stdev6[1] * stdev6[2] * -1
]
print(f'XXX min: {xxx_range[0]}')
print(f'XXX max: {xxx_range[1]}')

XXX min: 0.036
XXX max: -0.036
